<a href="https://colab.research.google.com/github/HerbertMariano/time-series-forecasting/blob/main/mancha_solar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mlp
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.neural_network import MLPRegressor
from skopt import BayesSearchCV
from sklearn.preprocessing import StandardScaler   

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-sunspots.csv')

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df['Month'] = pd.to_datetime(df['Month'])

In [ ]:
df

In [ ]:
def detect_outliers(df, col_name, threshold=3):
    mean = df[col_name].mean()
    std = df[col_name].std()
    outliers = np.where(np.abs(df[col_name]-mean) > (threshold*std),1,0)
    
    return outliers

In [ ]:
def create_dataset(df:pd.DataFrame, target:str, features:str, passado=12, futuro=3):
    X = y = []
    for i in range(passado, len(df)-futuro):
        X.append(df[features].iloc[i-passado:i].values)
        y.append(df[target].iloc[i:i+futuro].values)
    return X,y

In [ ]:
decom = seasonal_decompose(df['Sunspots'], model='additive', period=12)
df['seasonal'] = decom.seasonal

In [ ]:
df['outlier'] = detect_outliers(df,'Sunspots')
df['jan'] = np.where(df['Month'].dt.month == 1,1,0)
df['fev'] = np.where(df['Month'].dt.month == 2,1,0)
df['mar'] = np.where(df['Month'].dt.month == 3,1,0)
df['abr'] = np.where(df['Month'].dt.month == 4,1,0)
df['mai'] = np.where(df['Month'].dt.month == 5,1,0)
df['jun'] = np.where(df['Month'].dt.month == 6,1,0)
df['jul'] = np.where(df['Month'].dt.month == 7,1,0)
df['ago'] = np.where(df['Month'].dt.month == 8,1,0)
df['set'] = np.where(df['Month'].dt.month == 9,1,0)
df['out'] = np.where(df['Month'].dt.month == 10,1,0)
df['nov'] = np.where(df['Month'].dt.month == 11,1,0)
df['dez'] = np.where(df['Month'].dt.month == 12,1,0)

In [ ]:
scaler = StandardScaler()
df['Sunspots'] = scaler.fit_transform(df['Sunspots'].values.reshape(-1,1))
df['seasonal'] = scaler.fit_transform(df['seasonal'].values.reshape(-1,1))

In [ ]:
features = ['Sunspots','seasonal','outlier','jan','fev','mar','abr','mai','jun','jul','ago','set','out','nov','dez']
target = ['Sunspots']

In [ ]:
X,y = create_dataset(df=df,features=features,target=target,passado=12,futuro=3)

In [ ]:
mlp = MLPRegressor()

param_dist = {'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],'max_iter': [100,1000,10000],
              'activation': ['relu', 'tanh'],
              'solver': ['sgd', 'adam'],
              'alpha': np.logspace(-5, 3, 5),
              'learning_rate': ['constant','adaptive']}

In [ ]:
bayes_cv = BayesSearchCV(mlp,param_dist, n_iter=10, cv=5, n_jobs=-1)
bayes_cv.fit(X, y)
print("Best parameters found: ",bayes_cv.best_params_)